## Notebook para processar dados dos anos do enem e processa-los

In [1]:
import fitz
from typing import List
import re

## Para o ano de 2009

In [2]:
reader = fitz.open('data/raw/2019/caderno-de-questoes-1-dia-caderno-1-azul-aplicacao-regular.pdf')
# for line in reader[1].get_text().split('\n'):
#     print(line)
print(reader[1].get_text())

LC - 1º dia | Caderno 1 - AZUL - Página 2
LINGUAGENS, CÓDIGOS  
E SUAS TECNOLOGIAS
Questões de 01 a 45
Questões de 01 a 05 (opção inglês)
Questão 01 
5 Ways Pets Can Improve Your Health
A pet is certainly a great friend. After a difficult day, 
pet owners quite literally feel the love.
In fact, for nearly 25 years, research has shown that 
living with pets provides certain health benefits. Pets help 
lower blood pressure and lessen anxiety. They boost our 
immunity. They can even help you get dates.
Allergy Fighters: A growing number of studies have 
suggested that kids growing up in a home with “furred 
animals” will have less risk of allergies and asthma.
Date Magnets: Dogs are great for making love 
connections. Forget Internet matchmaking — a dog is a 
natural conversation starter.
Dogs for the Aged: Walking a dog or just caring for 
a pet — for elderly people who are able — can provide 
exercise and companionship.
Good for Mind and Soul: Like any enjoyable activity, 
playing with 

In [3]:
DAY_1_QUESTIONS = ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","48","49","50","51","52","53","54","55","56","57","58","59","60","61","62","63","64","65","66","67","68","69","70","71","72","73","74","75","76","77","78","79","80","81","82","83","84","85","86","87","88","89","90","91","92","93","94","95","96","97","98","99","100","101","102","103","104","105","106","107","108","109","110","111","112","113","114","115","116","117","118","119","120","121","122","123","124","125","126","127","128","129","130","131","132","133","134","135","136","137","138","139","140","141","142","143","144","145","146","147","148","149","150","151","152","153","154","155","156","157","158","159","160","161","162","163","164","165","166","167","168","169","170","171","172","173","174","175","176","177","178","179","180"]

In [31]:
def concat_lines_questions(
    text_page: str
):
    questions = []
    
    aux = []
    start_question= False
    for line in text_page.split('\n'):
        
        if line.lower().startswith("questão"):
            start_question = True
            if len(aux) != 0:
                questions.append(aux)
            aux = [line]
        elif start_question:
            aux.append(line)

    if len(aux) != 0:
        questions.append(aux)

    return questions

def concat_numbers(
    questions: List[List[str]]
):
    new_questions = []
    for question in questions:
        if question[0].lower().startswith("questão") and question[1].strip().isnumeric():
            new_question = [question[0] + " "+ question[1].strip()] + question[2:]
            new_questions.append(new_question)
        else:
            new_questions.append(question)
    return new_questions

def identify_alternatives(
    questions: List[List[str]]
):
    new_questions = []
    
    for question in questions:
        
        alternatives = []
        
        aux = ""
        index_alternatives = None
        for index in range(len(question)-1,-1,-1):
            
            aux = question[index] + " " + aux
            if question[index][0] in ["A","B","C","D","E","F","G"]:
                alternatives = [aux] + alternatives
                aux = ""
            if question[index][0] == "A":
                index_alternatives = index
                break
        
        new_questions.append({"description":question[:index_alternatives], "alternatives": alternatives})
        
    return new_questions
    
    
def ignore_lines(
    questions: List[List[str]]
):
    
    regexes = ["\*[A-Z0-9]*\*"]
    values = ['',' ']
    some = [
        'The English Enlightenment',
        'SILVA','São Paulo: Livraria',
        '2006. p. 169 (adaptado)',
        'LINGUAGENS, CÓDIGOS',
        'E SUAS TECNOLOGIAS',
        ' 01 a 05 (opção espanhol)',
        'de 01 a 45'
    ]
    
    new_questions = []
    for question in questions:
        question = list(filter(lambda x: not(any([re.fullmatch(r,x) for r in regexes])), question))
        question = list(filter(lambda x: not(any([v == x for v in values])), question))
        question = list(filter(lambda x: not(any([v in x for v in some])), question))
        new_questions.append(question)
    
    return new_questions
                 

def process_exam(
    path_exam: str
):
    reader = fitz.open(path_exam)
    
    questions = []
    for index, page in enumerate(reader):
        questions.extend(concat_lines_questions(page.get_text()))
    
    questions = ignore_lines(questions)
    questions = concat_numbers(questions)
    questions = identify_alternatives(questions)
    
    return questions
        


In [32]:
print(process_exam("data/raw/2019/caderno-de-questoes-1-dia-caderno-1-azul-aplicacao-regular.pdf")[0].keys())
for q in process_exam("data/raw/2019/caderno-de-questoes--2-dia-caderno-5-amarelo-aplicacao-regular.pdf"):
    for a in q['description']:
        print(a)
    print()

dict_keys(['description', 'alternatives'])
Questão 91 
Uma casa tem um cabo elétrico mal dimensionado, 
de resistência igual a 10 Ω, que a conecta à rede elétrica 
de 120 V. Nessa casa, cinco lâmpadas, de resistência 
igual a 200 Ω, estão conectadas ao mesmo circuito que 
uma televisão de resistência igual a 50 Ω, conforme 
ilustrado no esquema. A televisão funciona apenas com 
tensão entre 90 V e 130 V.
10 Ω
200 Ω
200 Ω
200 Ω
200 Ω
200 Ω
50 Ω
120 V
O número máximo de lâmpadas que podem ser ligadas 
sem que a televisão pare de funcionar é:

Questão 92 
O 2,4-dinitrofenol (DNP) é conhecido como 
desacoplador da cadeia de elétrons na mitocôndria e 
apresenta um efeito emagrecedor. Contudo, por ser 
perigoso e pela ocorrência de casos letais, seu uso 
como medicamento é proibido em diversos países, 
inclusive no Brasil. Na mitocôndria, essa substância 
captura, no espaço intermembranas, prótons (H+) 
provenientes da atividade das proteínas da cadeia 
respiratória, 
retornando-os 
à 
matri

In [ ]:
process_exam("data/raw/2019/caderno-de-questoes-1-dia-caderno-1-azul-aplicacao-regular.pdf")